In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):

    mask = np.zeros_like(img)

    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    cv2.fillPoly(mask, vertices, ignore_mask_color)

    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):

    # Creating a copy of the original image
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    # Drawing
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)

    return line_img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

def separate_lines(lines, img_shape):

    left_lines = []
    right_lines = []

    if lines is None:
        return [], []

    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2 == x1:
                continue  # Skiping vertical lines

            slope = (y2 - y1) / (x2 - x1)

            # Filter based on slope
            if abs(slope) < 0.5:  # Ignore lines with near zero slope
                continue

            if slope < 0:  # Negative slope, left lane
                left_lines.append(line)
            else:  # Positive slope, right lane
                right_lines.append(line)

    return left_lines, right_lines

def average_line(lines, img_shape):
    if len(lines) == 0:
        return None

    x_sum = 0
    y_sum = 0
    m_sum = 0

    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2 == x1:
                continue  # Skiping vertical lines

            x_sum += x1 + x2
            y_sum += y1 + y2
            m_sum += (y2 - y1) / (x2 - x1)

    x_avg = x_sum / (2 * len(lines))
    y_avg = y_sum / (2 * len(lines))
    m_avg = m_sum / len(lines)

    # y = mx + b
    b = y_avg - m_avg * x_avg

    # top and bottom y-coordinates
    y1 = img_shape[0]  # Bottom
    y2 = int(img_shape[0] * 0.6)  # Top

    #  x-coordinates
    x1 = int((y1 - b) / m_avg)
    x2 = int((y2 - b) / m_avg)

    return np.array([[x1, y1, x2, y2]])

def process_frame(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) if len(image.shape) == 3 else image

    height, width = image.shape[:2]

    gray = grayscale(image_rgb)

    blurred = gaussian_blur(gray, kernel_size=5)

    edges = canny(blurred, low_threshold=50, high_threshold=150)

    # Region of Interest (lanes)
    roi_vertices = np.array([
        [(0, height), (width / 2 - 50, height / 2 + 50),
         (width / 2 + 50, height / 2 + 50), (width, height)]
    ], dtype=np.int32)

    masked_edges = region_of_interest(edges, roi_vertices)

    lines = hough_lines(masked_edges, rho=1, theta=np.pi/180,
                       threshold=20, min_line_len=20, max_line_gap=300)

    # Separating and averaging left and right lane lines
    left_lines, right_lines = separate_lines(lines, image.shape)

    line_img = np.zeros((height, width, 3), dtype=np.uint8)

    # Left lane
    if left_lines:
        avg_left_line = average_line(left_lines, image.shape)
        if avg_left_line is not None:
            cv2.line(line_img,
                     (avg_left_line[0][0], avg_left_line[0][1]),
                     (avg_left_line[0][2], avg_left_line[0][3]),
                     [255, 0, 0], 5)

    # Right lane
    if right_lines:
        avg_right_line = average_line(right_lines, image.shape)
        if avg_right_line is not None:
            cv2.line(line_img,
                     (avg_right_line[0][0], avg_right_line[0][1]),
                     (avg_right_line[0][2], avg_right_line[0][3]),
                     [255, 0, 0], 5)

    result = cv2.addWeighted(image, 0.8, line_img, 1, 0)

    return result

def process_video(input_path, output_path):
    clip = VideoFileClip(input_path)
    output_clip = clip.fl_image(process_frame)
    output_clip.write_videofile(output_path, audio=False)

def visualize_pipeline(image_path):
    image = cv2.imread(image_path)

    result = process_frame(image)

    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.title('Lane Detection Result')
    plt.axis('off')
    plt.show()

if __name__ == "__main__":
    input_video = "Driving_Along_An_Empty_Road_Royalty_Free_4K_Stock_Video_Footage.mp4"
    output_video = "output_result_U2110063.mp4"

    print("Processing...")
    process_video(input_video, output_video)
    print("Processing complete!")

Processing...
Moviepy - Building video output_result_U2110063.mp4.
Moviepy - Writing video output_result_U2110063.mp4



t: 100%|█████████▉| 1233/1235 [02:28<00:00,  8.47it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file Driving_Along_An_Empty_Road_Royalty_Free_4K_Stock_Video_Footage.mp4, 2764800 bytes wanted but 0 bytes read,at frame 1232/1235, at time 41.11/41.18 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|██████████| 1235/1235 [02:28<00:00, 10.01it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file Driving_Along_An_Empty_Road_Royalty_Free_4K_Stock_Video_Footage.mp4, 2764800 bytes wanted but 0 bytes read,at frame 1234/1235, at time 41.17/41.18 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready output_result_U2110063.mp4
Processing complete!
